In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import random
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv('digit_data.csv')

In [ ]:
df.head(5)

In [ ]:
numpy_array = df.to_numpy()

In [ ]:
num_of_rows, num_of_columns = numpy_array.shape

In [ ]:
np.random.shuffle(numpy_array)

In [ ]:
numpy_array.shape

In [ ]:
# train_data = numpy_array[1000:]

train_data, test_data, train_data_label, test_data_label = train_test_split(numpy_array[:,1:], numpy_array[:, 0],test_size=0.023, shuffle=True, random_state=43)

In [ ]:
train_data.shape

In [ ]:
test_data, train_data = test_data.T, train_data.T

In [ ]:
test_data.shape, train_data.shape

In [ ]:
test_data_label = test_data[0]

In [ ]:
test_data_label.shape

In [ ]:
train_data_label

In [ ]:
train_data_label = train_data[0]

In [ ]:
train_data_label.shape

In [ ]:
Y = train_data_label

In [ ]:
test_data[100]

In [ ]:
# test_data, train_data = test_data[1:], train_data[1:]

In [ ]:
test_data.shape, train_data.shape

In [ ]:
data = []
count = 0

In [ ]:
pixel_value = {i:0 for i in range(0,256)}

In [ ]:
pixel_value

In [ ]:
test_data.shape

In [ ]:
for _ in test_data.T:
    for i in np.unique(_):
        if i in pixel_value:
            pixel_value[i] +=1

In [ ]:
pixel_value

In [ ]:
pixel_value

In [ ]:
pixels = list(pixel_value.keys())
values = list(pixel_value.values())



In [ ]:
values

In [ ]:
plt.bar(pixels, values)

In [ ]:
data_range = [i for i in range(0,260,5)]
data_range

In [ ]:
pixels_to_plot = {i:pixel_value[i] for i in data_range}

In [ ]:
pixels_to_plot
x_axis = list(pixels_to_plot.keys())
y_axis = list(pixels_to_plot.values())

In [ ]:
plt.bar(x_axis,y_axis)

In [ ]:
num_of_columns

In [ ]:
# plt.bar(data, height=2)

In [ ]:
test_data = test_data/255
train_data = train_data/255

In [ ]:
train_data_label[89:200]

In [ ]:
A0 = train_data

In [ ]:
# random_index = random.randint(0,41000)
# print(random_index)
# plot_images = train_data.T
# plt.imshow(plot_images[random_index].reshape(28,28))
# plt.show()

In [ ]:
def weight_initilization(x_value, y_value):
    # generated = np.random.gumbel(loc=5,size=(x_value,y_value))
    # generated = np.random.uniform(-0.05, 0.05,size=(x_value,y_value))
    generated  = np.random.exponential(scale=0.5, size=(x_value, y_value))
    # return  np.clip(generated, -0.5, 0.5)
    return generated

In [ ]:
def initialize_parameters():
    w1 = weight_initilization(10, 784)
    w2 = weight_initilization(10,10)
    b1 = np.zeros((10, 1))  
    b2 = np.zeros((10, 1))
    return w1, b1, w2, b2

In [ ]:
w1, b1, w2, b2 = initialize_parameters()

In [ ]:
# plt.hist(w2.ravel(), bins=100, histtype='bar')

In [ ]:
# plotting the data of the weights initilaized
def weights(data):
    plt.hist(data.ravel(), bins=100, histtype='bar')
    unique, frequency = np.unique(data,return_counts = True)
    count = np.asarray((unique, frequency))
    print(count.T)
    print(f'the sum is {sum(count.T)}')

In [ ]:
# weights(w2)

In [ ]:
# weights(w1)

In [ ]:
# ReLU -> if x < 0 == 0, else x
def Relu(input_vector):
    return np.maximum(input_vector, 0)

In [ ]:
x =  [-1.3, -5.1, 2.2, 0.7, 1.1]
Relu(x)

In [ ]:
# change of x w.r.t.x is 1
def relu_derivative(Z):
    return np.where(Z > 0, 1, 0)

In [ ]:
def softmax(input_vector):
    exp_values = np.exp(input_vector - np.max(input_vector))
    return exp_values / np.sum(exp_values)

In [ ]:
x =  [1.3, 5.1, 2.2, 0.7, 1.1]
softmax(x)

In [ ]:
# 17 -> equations for forward propoagation
def forward_propagation(w1, b1, w2, b2, train_data):
    Z1 = np.dot(w1,train_data) + b1
    A1 = Relu(Z1)
    Z2 = np.dot(w2,A1) + b2
    A2 = softmax(Z2)
    return  Z1, A1, Z2, A2

In [ ]:
Z1, A1, Z2, A2 = forward_propagation(w1, b1, w2, b2, train_data)

In [ ]:
Z1.shape, A1.shape, Z2.shape, A2.shape

In [ ]:
# 18 -> one hot encoded class label
def one_hot_encoded_data (Y):
    one_hot_encoded_class_label = pd.get_dummies(Y, dtype=int)
    return one_hot_encoded_class_label

In [ ]:
# 19 - > dw1, db1, db2, dw2
def backpropagation(Z1, A1, Z2, A2, w1, w2, train_data, train_data_label):
    A0 = train_data
    m = train_data.shape[1]
    
    dz2 = A2 - train_data_label  

   
    dw2 = (1/m) * np.dot(dz2, A1.T) 
    db2 = (1/m) * np.sum(dz2, axis=1, keepdims=True)  

    dz1 = np.dot(w2.T, dz2) * relu_derivative(Z1)  

    dw1 = (1/m) * np.dot(dz1, A0.T)  
    db1 = (1/m) * np.sum(dz1, axis=1, keepdims=True)  

    return dw1, db1, dw2, db2

In [ ]:
# 20 -> updating the parameters
# -> learning rate = alpha 
def update_parameters(w1, b1, w2, b2, dw1, db1, dw2, db2, alpha):
    # alpha = 0.00001
    w1 = w1 - alpha * dw1
    b1 = b1 - alpha * db1
    w2 = w2 - alpha * dw2
    b2 = b2 - alpha * db2
    return w1, b1, w2, b2

In [ ]:
def obtain_predictions(vector): 
    return np.argmax(vector, axis=0)

In [ ]:
def accuracy_score(y_prediction, y_actual):  
    num_samples = y_actual.shape[0]

    if y_prediction.ndim ==2:
        pred_label = obtain_predictions(y_prediction)
    else:
        pred_label = y_prediction
        
    if y_actual.ndim == 2:
        actual_label = obtain_predictions(y_actual)
    else:
        actual_label = y_actual
    
    num_correct = np.sum(pred_label == actual_label)

    return num_correct / num_samples

In [ ]:
def gradient_descent(X_train, y_train, learning_rate, num_iterations):
    W1, b1, W2, b2 = initialize_parameters()

    accuracy_list = []

    for i in range(num_iterations+1):
        Z1, A1, Z2, A2 = forward_propagation(W1, b1, W2, b2, X_train)
        dW1, db1, dW2, db2 = backpropagation(Z1, A1, Z2, A2, W1, W2, X_train, y_train)
        W1, b1, W2, b2 = update_parameters(W1, b1, W2, b2, dW1, db1, dW2, db2, learning_rate)

        if i%10==0:
            accuracy = accuracy_score(A2,y_train)
            accuracy_list.append((i,accuracy))
            print(f"Iteration: {i} \t Accuracy: {accuracy:.5f}")

    return W1, b1, W2, b2, accuracy_list

In [ ]:
def make_prediction(X_test, W1, b1, W2, b2):
    
    _, _, _, A2 = forward_propagation(W1, b1, W2, b2, X_test)
    
    pred = obtain_predictions(A2)
    
    return pred


In [ ]:
def plot_image(image):
    X_test_2d = image.reshape(28,-1)
    print(X_test_2d.shape)
    plt.imshow(X_test_2d) 
    plt.axis('off')
    plt.show()

In [ ]:
w1, b1, w2, b2, accuracy_list = gradient_descent(train_data, train_data_label, learning_rate=0.00000000001, num_iterations=100)

In [ ]:
w1

In [ ]:
iterations, accuracies = zip(*accuracy_list)

plt.figure(figsize=(10, 6))
plt.plot(iterations, accuracies, marker='o', linestyle='-', color='b')
plt.title('Accuracy vs. Iteration')
plt.xlabel('Iteration')
plt.ylabel('Accuracy')
plt.grid(False)
plt.show()


In [ ]:
def test_predictions(W1, b1, W2, b2, X_test, y_test):
    test = make_prediction(X_test, W1, b1, W2, b2)
    image = X_test.T
    # print(test.shape)
    accuracy = accuracy_score(test, y_test)
    print(f"Accuracy:{accuracy}")
    # y_test_label = obtain_predictions(y_test)
    # print(y_test.shape)
    for i in range(10):
        # print(X_test[i], y_test[i])
        # print(image[i].shape)
        print(test[i])
        plot_image(image[i])

In [ ]:
# def test_predictions(w1, b1, w2, b2, test_data, test_data_label):
#     image = test_data.T
#     y_test = make_prediction(test_data, w1, b1, w2, b2)
#     y_test_label = obtain_predictions(test_data_label)
#     for i in range(10):
#         print(test_data[i], y_test_label[i])
#         print(image[i].shape)
#         plot_image(image[i])


In [ ]:
test_data_label.shape

In [ ]:
test_predictions(w1, b1, w2, b2, test_data, test_data_label)